In [ ]:
# Cell 1: Imports
import pandas as pd
import random
from agent import get_response
from tqdm.notebook import tqdm  # Progress bar

In [ ]:
from config import client

def get_response(user_input: str, temperature=0.9, top_p=0.8):
    try:
        completion = client.chat.completions.create(
            model="openai/gpt-oss-20b",
            messages=[
                {"role": "system", "content": "Act as a creative marketing assistant."},
                {"role": "user", "content": user_input}
            ],
            temperature=temperature,
            top_p=top_p,
            max_completion_tokens=4096,
            stream=True,
        )
        response = ""
        for chunk in completion:
            content = chunk.choices[0].delta.content or ""
            response += content
            yield content
    except Exception as e:
        yield f"⚠️ Error: {str(e)}"

In [ ]:
# Cell 2: Generate random parameter sets
prompts = [
    "Generate marketing campaign for a tech startup",
    "Generate marketing campaign for a fashion brand",
    "Generate marketing campaign for an eco-product",
    "Generate marketing campaign for a new SaaS platform",
    "Generate marketing campaign for a health & wellness product",
]

# Generate 20 experiments
data = []
for i in range(1):
    data.append({
        "prompt": random.choice(prompts),
        "temperature": round(random.uniform(0.5, 1.0), 2),
        "top_p": round(random.uniform(0.7, 1.0), 2),
        "output": ""
    })

# Save to Excel for reference
df = pd.DataFrame(data)
print(df)
#df.to_excel("experiment_parameters.xlsx", index=False)
#df.head()

In [ ]:
# Cell 3: Run experiments with your agent
df = pd.read_excel("experiment_parameters.xlsx")

# Collect results
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Running experiments"):
    prompt = row["prompt"]
    temperature = row["temperature"]
    top_p = row["top_p"]

    # Get response from your agent with custom temperature & top_p
    response_text = ""
    for chunk in get_response(prompt, temperature=temperature, top_p=top_p):
        response_text += chunk

    # Save output in dataframe
    df.at[idx, "output"] = response_text

In [ ]:
# Cell 4: Save results
df.to_excel("experiment_results.xlsx", index=False)
print("Results saved to 'experiment_results.xlsx'")
df.head()